In [171]:
import pandas as pd
import re
import numpy as np
df = pd.read_csv('LikelihoodRatiosDataFrame_GLR/DxMxLR.csv', index_col=0)

In [172]:
df.head()

,Association,Dx Name,Mx Name,Prevalence,Import,LR (+),LR (-)
0,"362, 59",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM RECURRENT ATTACK <S> HX,0.5,2.0,1.500000,0.5000
1,"362, 62",PEPTIC ULCER,ABDOMEN PAIN EPIGASTRIUM SEASONAL HX,0.5,2.0,0.666667,1.2000
2,"362, 64",PEPTIC ULCER,ABDOMEN PAIN EXACERBATION WITH ALCOHOL,0.5,2.0,0.666667,2.0000
3,"362, 198",PEPTIC ULCER,AGE 16 TO 25,0.5,3.0,0.000100,-0.0003
4,"362, 199",PEPTIC ULCER,AGE 26 TO 55,0.5,3.0,0.000100,-0.0001


In [173]:
Mx_list = ['+HEPATOMEGALY PRESENT', '-JAUNDICE FAMILY HX', '-ABDOMEN PAIN COLICKY']
Dx_list = pd.DataFrame(columns=['Rank', 'Dx', 'Score', 'Importance_List'])
Dx_list['Dx']=pd.Series((list(set(df['Dx Name']))))
Dx_list['Importance_List']=pd.Series( [ [] for i in range(10) ])

In [174]:
for i,r in Dx_list.iterrows():
    #r['Dx'] = disease
    print(r['Dx'])
    score = 1
    for mx in Mx_list: 
        if re.match('[+]', mx): 
            mx = mx.strip('+')
            q = df.loc[df['Dx Name'] == r['Dx']]
            q = q.loc[df['Mx Name'] == mx]
            im = q['Import'].values
            p = q['LR (+)'].values
            print(p)
            if p.size == 0: continue #don't count NAs
            else:
                score = score * p
                Dx_list.loc[i,'Importance_List'].append(float(im))
        elif re.match('[-]', mx): 
            mx = mx.strip('-')
            s = df.loc[df['Dx Name'] == r['Dx']]
            s = s.loc[df['Mx Name'] == mx]
            im = s['Import'].values
            m = s['LR (-)'].values
            print(m)
            if m.size == 0: continue #don't count NAs
            else: 
                score = score * m
                Dx_list.loc[i,'Importance_List'].append(float(im))
        else: print("Mx ", mx, "is not positive or negative. Please indicate presence or absence of mx.")
    if not Dx_list.loc[i,'Importance_List']: 
        Dx_list.loc[i, 'Score'] = 0
    else:
        Dx_list.loc[i, 'Score'] = float(score)

PANCREATITIS CHRONIC
[]
[]
[]
CHOLEDOCHOLITHIASIS
[ 24.75]
[ 0.75765306]
[ 0.50381679]
ANISAKIASIS
[]
[]
[ 0.7500075]
PEPTIC ULCER
[]
[]
[]
GASTRIC LYMPHOMA
[]
[]
[]
GASTRITIS GIANT HYPERTROPHIC <MENETRIERS>
[]
[]
[]
PYLORIC OBSTRUCTION
[]
[]
[ 0.75007502]
PANCREATITIS ACUTE
[]
[]
[]
GASTROINTESTINAL SARCOIDOSIS
[]
[]
[ 0.75007502]
GASTRIC CARCINOMA
[]
[]
[]


In [175]:
Dx_list = Dx_list.sort_values(by="Score", ascending= False)

In [176]:
rank = 1
for i,r in Dx_list.iterrows():
    Dx_list.loc[i, 'Rank'] = rank
    rank = rank + 1
Dx_list.set_index('Rank', inplace=True)
Dx_list

,Dx,Score,Importance_List
Rank,,,
1,CHOLEDOCHOLITHIASIS,9.44753,"[2.0, 2.0, 3.0]"
2,PYLORIC OBSTRUCTION,0.750075,[3.0]
3,GASTROINTESTINAL SARCOIDOSIS,0.750075,[3.0]
4,ANISAKIASIS,0.750008,[3.0]
5,PANCREATITIS CHRONIC,0,[]
6,PEPTIC ULCER,0,[]
7,GASTRIC LYMPHOMA,0,[]
8,GASTRITIS GIANT HYPERTROPHIC <MENETRIERS>,0,[]
9,PANCREATITIS ACUTE,0,[]
